# Heading of the problem statement

One liner descripition about the code it is going to do

**Problem Statement**

Talk about the problem statement we are going to solve.
- subpoint1
- subpoint2




### Part 1 - Describe about the steps taken in this section i,e part-1

In [1]:
#import the libraries
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")
import seaborn as sns

### Part 2 - Describe about the steps taken in this section i,e part-2

Give some comments about the below code is going to do
 

In [2]:
#Check the datatypes of all the columns of the dataframe
lendingDF = pd.read_csv('loan.csv')
lendingDF.dtypes

id                              int64
member_id                       int64
loan_amnt                       int64
funded_amnt                     int64
funded_amnt_inv               float64
                               ...   
tax_liens                     float64
tot_hi_cred_lim               float64
total_bal_ex_mort             float64
total_bc_limit                float64
total_il_high_credit_limit    float64
Length: 111, dtype: object

In [3]:
lendingDF.head(5)

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
0,1077501,1296599,5000,5000,4975.0,36 months,10.65%,162.87,B,B2,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
1,1077430,1314167,2500,2500,2500.0,60 months,15.27%,59.83,C,C4,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
2,1077175,1313524,2400,2400,2400.0,36 months,15.96%,84.33,C,C5,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
3,1076863,1277178,10000,10000,10000.0,36 months,13.49%,339.31,C,C1,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
4,1075358,1311748,3000,3000,3000.0,60 months,12.69%,67.79,B,B5,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN


In [4]:
lendingDF.shape

(39717, 111)

In [5]:
columnheaders = columnheaders = list(lendingDF.columns.values)
print('Column names are ', columnheaders)

Column names are  ['id', 'member_id', 'loan_amnt', 'funded_amnt', 'funded_amnt_inv', 'term', 'int_rate', 'installment', 'grade', 'sub_grade', 'emp_title', 'emp_length', 'home_ownership', 'annual_inc', 'verification_status', 'issue_d', 'loan_status', 'pymnt_plan', 'url', 'desc', 'purpose', 'title', 'zip_code', 'addr_state', 'dti', 'delinq_2yrs', 'earliest_cr_line', 'inq_last_6mths', 'mths_since_last_delinq', 'mths_since_last_record', 'open_acc', 'pub_rec', 'revol_bal', 'revol_util', 'total_acc', 'initial_list_status', 'out_prncp', 'out_prncp_inv', 'total_pymnt', 'total_pymnt_inv', 'total_rec_prncp', 'total_rec_int', 'total_rec_late_fee', 'recoveries', 'collection_recovery_fee', 'last_pymnt_d', 'last_pymnt_amnt', 'next_pymnt_d', 'last_credit_pull_d', 'collections_12_mths_ex_med', 'mths_since_last_major_derog', 'policy_code', 'application_type', 'annual_inc_joint', 'dti_joint', 'verification_status_joint', 'acc_now_delinq', 'tot_coll_amt', 'tot_cur_bal', 'open_acc_6m', 'open_il_6m', 'open

In [6]:
rowcounts = lendingDF.shape[0]

null_columnnames=[]
for x in columnheaders:
    null_percentage = (lendingDF[x].isnull().sum()/rowcounts) * 100
    if null_percentage > 40:
        print("count of nulls in",x," ::", sum)
        null_columnnames.append(x)

lendingDF.drop(null_columnnames,axis=1,inplace=True)
lendingDF.shape

count of nulls in mths_since_last_delinq  :: <built-in function sum>
count of nulls in mths_since_last_record  :: <built-in function sum>
count of nulls in next_pymnt_d  :: <built-in function sum>
count of nulls in mths_since_last_major_derog  :: <built-in function sum>
count of nulls in annual_inc_joint  :: <built-in function sum>
count of nulls in dti_joint  :: <built-in function sum>
count of nulls in verification_status_joint  :: <built-in function sum>
count of nulls in tot_coll_amt  :: <built-in function sum>
count of nulls in tot_cur_bal  :: <built-in function sum>
count of nulls in open_acc_6m  :: <built-in function sum>
count of nulls in open_il_6m  :: <built-in function sum>
count of nulls in open_il_12m  :: <built-in function sum>
count of nulls in open_il_24m  :: <built-in function sum>
count of nulls in mths_since_rcnt_il  :: <built-in function sum>
count of nulls in total_bal_il  :: <built-in function sum>
count of nulls in il_util  :: <built-in function sum>
count of nul

(39717, 54)

In [7]:
lendingDF.isnull().sum()

id                                0
member_id                         0
loan_amnt                         0
funded_amnt                       0
funded_amnt_inv                   0
term                              0
int_rate                          0
installment                       0
grade                             0
sub_grade                         0
emp_title                      2459
emp_length                     1075
home_ownership                    0
annual_inc                        0
verification_status               0
issue_d                           0
loan_status                       0
pymnt_plan                        0
url                               0
desc                          12940
purpose                           0
title                            11
zip_code                          0
addr_state                        0
dti                               0
delinq_2yrs                       0
earliest_cr_line                  0
inq_last_6mths              

#### Comments

In [8]:
#Check the number of null values in the columns

Additional comments